In [11]:
from matplotlib import pyplot as plt, cm
import numpy as np
import glob, os
import h5py
import ntpath
import SimpleITK as sitk
import cv2


%matplotlib inline

DATA_DIR = "C:/Users/Kevin/Desktop/patient data/original/train"
    
patients = glob.glob(os.path.join(DATA_DIR, '*')) # Get the folder names for the patients - for training

DATA_DIR_TESTING = "C:/Users/Kevin/Desktop/patient data/original/test"

patients_testing = glob.glob(os.path.join(DATA_DIR_TESTING, '*'))

def getImageTensor(patientDirectory):
    """
    Helper function for injesting all of the DICOM files for one study into a single tensor
    
    input: 'patientDirectory', the directory where the DICOMs for a single patient are stored
    outputs:
            imgTensor = a flattened numpy array (1, C*H*W*D)
            C = number of channels per pixel (1 for MR, CT, and Xray)
            H = number of pixels in height
            W = number of pixels in width
            D = number of pixels in depth
    """
    
    imgTensor = []
    reader = sitk.ImageSeriesReader()  # Set up the reader object
    
    ##for imgTensor
    for patientDirectory in patients[:1]:
        # Now get the names of the DICOM files within the directory
        filenamesDICOM = reader.GetGDCMSeriesFileNames(patientDirectory)
        reader.SetFileNames(filenamesDICOM)
        # Now execute the reader pipeline
        patientObject = reader.Execute() 

        imgTensor = sitk.GetArrayFromImage(patientObject)
        
    for patientDirectory in patients[1:]:
        # Now get the names of the DICOM files within the directory
        filenamesDICOM = reader.GetGDCMSeriesFileNames(patientDirectory)
        reader.SetFileNames(filenamesDICOM)
        # Now execute the reader pipeline
        patientObject = reader.Execute()

        C = patientObject.GetNumberOfComponentsPerPixel() # There is just one color channel in the DICOM for CT and MRI
        H = patientObject.GetHeight()  # Height in pixels
        W = patientObject.GetWidth()   # Width in pixels
        D = patientObject.GetDepth()  # Depth in pixels

        imgTensor = np.append(imgTensor, sitk.GetArrayFromImage(patientObject), axis=0)
        
    ##for imgTensorCSM
    

    ##preparing for CSM coil approximation 
    imgTensorCSM = []
    number_of_coils = 12
    
    for img in imgTensor[:1]:
        temp_arr = []
        img = np.expand_dims(img, axis = 0)
        
        #duplicates image and adds it into temp. array
        for i in range(number_of_coils):
            if i == 0:
                temp_arr = img

            else:
                temp_arr = np.append(temp_arr, img, axis = 0)
                
        temp_arr = np.expand_dims(temp_arr, axis = 0)
        
        #need to do this. 
        #If not, error and squeezing of array will happen when imcompatible size is detected during appending
        imgTensorCSM = temp_arr
        
    for img in imgTensor[1:]:
        temp_arr = []
        img = np.expand_dims(img, axis = 0)
        
        #duplicates image and adds it into temp. array
        for i in range(number_of_coils):
            if i == 0:
                temp_arr = img

            else:
                temp_arr = np.append(temp_arr, img, axis = 0)
                
        temp_arr = np.expand_dims(temp_arr, axis = 0)
        
        #need to do this. 
        #If not, error and squeezing of array will happen when imcompatible size is detected during appending
        imgTensorCSM = np.append(imgTensorCSM, temp_arr, axis = 0)
               
    
    return imgTensor, imgTensorCSM, C, H, W, D

imgTensor, imgTensorCSM, original_C, original_H, original_W, original_D = getImageTensor(patients)

print(imgTensor.shape)
print(imgTensorCSM.shape)

(368, 512, 512)
(368, 12, 512, 512)
